# Тестовое задание, Рассадин Ярослав

## Описание задачи

Необходимо создать следующие таблицы и наполнить их не менее чем 1 миллионом строк:

- таблица "users" с полями: id, name, email, created_at
- таблица "orders" с полями: id, user_id, total_price, created_at
- таблица "order_items" с полями: id, order_id, product_name, price, quantity

Также необходимо написать запросы, используя ClickHouse:

1.	Найти общее количество заказов каждого пользователя, который сделал более 10 заказов.
2.	Найти средний размер заказа для каждого пользователя за последний месяц.
3.	Найти средний размер заказа за каждый месяц в текущем году и сравнить его с средним размером заказа за соответствующий месяц в прошлом году.
4.	Найти 10 пользователей, у которых наибольшее количество заказов за последний год, и для каждого из них найти средний размер заказа за последний месяц.

## Подготовка данных для формирования запросов

In [1]:
from clickhouse_driver import Client
import random
import string
from datetime import datetime, timedelta
import pandas as pd

### Подключение к локальной базе

In [2]:
client = Client(host='localhost')

client.execute('CREATE DATABASE IF NOT EXISTS test_db')
client.execute('USE test_db')

[]

In [19]:
# Версия ClickHouse
client.execute(f'SELECT version()')

[('22.1.3.7',)]

In [3]:
# Очистка таблиц после тестовых запусков
tables = client.execute(f'SHOW TABLES FROM test_db')

for table in tables:
    table_name = table[0]
    client.execute(f'DROP TABLE IF EXISTS test_db.{table_name}')
    print(f"Таблица {table_name} успешно удалена.")

Таблица order_items успешно удалена.
Таблица orders успешно удалена.
Таблица users успешно удалена.


### Создание таблиц 

In [4]:
client.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id UInt64,
        name String,
        email String,
        created_at DateTime
    ) ENGINE = MergeTree()
    ORDER BY id
    ''')

client.execute('''
    CREATE TABLE IF NOT EXISTS orders (
        id UInt64,
        user_id UInt64,
        total_price Float64,
        created_at DateTime
    ) ENGINE = MergeTree()
    ORDER BY id
    ''')

client.execute('''
    CREATE TABLE IF NOT EXISTS order_items (
        id UInt64,
        order_id UInt64,
        product_name String,
        price Float64,
        quantity UInt32
    ) ENGINE = MergeTree()
    ORDER BY id
    ''')

[]

### Заполнение таблиц данными 

#### Генерация уникальных ID для пользователей


In [5]:
def generate_unique_user_ids(n):
    ids = set()
    while len(ids) < n:
        ids.add(random.randint(1000000000, 9999999999)) 
    return list(ids)

#### Таблица users 

Создана таблица **users**, состоящая из 1 миллиона строк, где:
- id - случайное число от 1000000000 до 9999999999,
- name - строка из 10 случайных заглавных и строчных букв,
- email - формируется на основе имени только из строчных букв и добававляется домен mail.ru,
- created_at - дата создания генерируется случайно в пределах 3 последних лет.

In [6]:
start_time = datetime.now()

num_users = 1000000
user_ids = generate_unique_user_ids(num_users)
users = []

for i, user_id in enumerate(user_ids):
    name = ''.join(random.choices(string.ascii_letters, k=10))
    email = f'{name.lower()}@mail.ru'
    created_at = datetime.now() - timedelta(days=random.randint(0, 3 * 365))  # случайная дата за 3 года
    users.append((user_id, name, email, created_at))

client.execute('INSERT INTO users (id, name, email, created_at) VALUES', users)

end_time = datetime.now()
print("Execution time:", end_time - start_time)

Execution time: 0:00:21.548088


In [7]:
row_count = client.execute('SELECT COUNT(*) FROM users')
print("Количество строк в таблице users:", row_count[0][0])

Количество строк в таблице users: 1000000


#### Таблица orders

In [8]:
start_time = datetime.now()

orders = []
order_counts = {}

# Код ниже заполняет таблицу orders по следующим правилам:

# 39% пользователей имеют 1 заказ
# 30% пользователей имеют от 2 до 3 заказов
# 20% пользователей имеют от 3 до 5 заказов
# 10% пользователей имеют от 5 до 10 заказов    
# 1% пользователей имеют от 10 до 50 заказов


for user_id in user_ids:
    rand_value = random.uniform(0, 1)  
    
    if rand_value < 0.39:  # 39%
        order_count = 1
    elif rand_value < 0.69:  # 30%
        order_count = random.randint(2, 3)
    elif rand_value < 0.89:  # 20%
        order_count = random.randint(3, 5)
    elif rand_value < 0.99:  # 10%
        order_count = random.randint(5, 10)
    else:  # 1%
        order_count = random.randint(10, 50)

    order_counts[user_id] = order_count

for user_id, count in order_counts.items():
    for _ in range(count):
        total_price = round(random.uniform(10, 500), 2)  

        days_in_two_years = 3 * 365 # генерация записей за последние 3 года
        random_days = random.randint(0, days_in_two_years)
        created_at = datetime.now() - timedelta(days=random_days) 
        
        orders.append((len(orders) + 1, user_id, total_price, created_at))  # уникальный id заказов
client.execute('INSERT INTO orders (id, user_id, total_price, created_at) VALUES', orders)

end_time = datetime.now()
print("Execution time:", end_time - start_time)

Execution time: 0:00:44.880076


In [9]:
row_count = client.execute('SELECT COUNT(*) FROM orders')
print("Количество строк в таблице orders:", row_count[0][0])

Количество строк в таблице orders: 2988164


#### Таблица order_items

In [10]:
start_time = datetime.now()

order_items = []

for order in orders:
    order_id = order[0]
    num_items = random.randint(1, 5)  
    
    for _ in range(num_items):
        product_name = f"Product{random.randint(1, 100)}"
        price = round(random.uniform(1, 50000), 2)
        quantity = random.randint(1, 10)  
        order_items.append((len(order_items) + 1, order_id, product_name, price, quantity))

client.execute('INSERT INTO order_items (id, order_id, product_name, price, quantity) VALUES', order_items)

end_time = datetime.now()
print("Execution time:", end_time - start_time)

Execution time: 0:01:25.253057


In [11]:
row_count = client.execute('SELECT COUNT(*) FROM order_items')
print("Количество строк в таблице order_items:", row_count[0][0])

Количество строк в таблице order_items: 8967328


## Написание запросов

### Общее количество заказов каждого пользователя, который сделал более 10 заказов

In [12]:
data, columns = client.execute(
    '''
    SELECT 
        u.id AS user_id,
        u.name AS user_name,
        COUNT(o.id) AS total_orders
    FROM users u
    JOIN orders o ON u.id = o.user_id
    GROUP BY u.id, u.name
    HAVING COUNT(o.id) > 10
    ORDER BY total_orders DESC;
    ''',
    with_column_types=True)

column_names = [column[0] for column in columns]
df = pd.DataFrame(data, columns=column_names)
display(df)

,user_id,user_name,total_orders
0,6215230195,UUALUgyegG,50
1,1904095594,XWzgHGUjYf,50
2,1458298847,dpboSpsWYB,50
3,2796481078,lqnxhehoLT,50
4,7575851004,uttiTdEKtq,50
...,...,...,...
9779,3164204534,DKeocPcZSI,11
9780,7412547215,mhoOfwxmXn,11
9781,3970052464,kpOOMsyjDC,11
9782,1394748314,kPScYGRMEY,11


### Найти средний размер заказа для каждого пользователя за последний месяц.

In [24]:
data, columns = client.execute(
    '''
    SELECT 
        u.id AS user_id,
        u.name AS user_name,
        AVG(o.total_price) AS avg_order_size
    FROM users AS u
    JOIN orders AS o ON u.id = o.user_id
    WHERE o.created_at >= now() - INTERVAL 1 MONTH
    GROUP BY u.id, u.name
    ORDER BY avg_order_size DESC;
    ''',
    with_column_types=True)

column_names = [column[0] for column in columns]
df = pd.DataFrame(data, columns=column_names)
display(df)

,user_id,user_name,avg_order_size
0,7758823493,DvHvPWBYUJ,500.00
1,8159094944,AyugyEBHlD,499.99
2,3360856058,ZvwTQxcwcq,499.99
3,8800542876,lmrZtjjmvY,499.99
4,4475678889,fNerYgzLsc,499.98
...,...,...,...
80150,8139395631,uOZAoxBycz,10.02
80151,1491612708,UCxIkrwOlB,10.02
80152,4688207173,ggnsWdQrxv,10.02
80153,7393252052,WjWcyCRbgt,10.02


### Средний размер заказа за каждый месяц в текущем году и средний размер заказа за соответствующий месяц в прошлом году

In [22]:
data, columns = client.execute(
    '''
    WITH 
        avg_2024 AS (SELECT 
            toMonth(created_at) AS month,
            AVG(total_price) AS avg_order_size_2024
        FROM orders
        WHERE 
            created_at >= '2024-01-01' AND created_at < '2025-01-01'
        GROUP BY month),
        
        avg_2023 AS (SELECT 
            toMonth(created_at) AS month,
            AVG(total_price) AS avg_order_size_2023
        FROM orders
        WHERE 
            created_at >= '2023-01-01' AND created_at < '2024-01-01'
        GROUP BY month)
        
    SELECT 
        toMonth(toDate('2024-01-01') + INTERVAL month-1 MONTH) AS month,
        avg_2024.avg_order_size_2024,
        avg_2023.avg_order_size_2023,
        (avg_2024.avg_order_size_2024 - avg_2023.avg_order_size_2023) AS difference
    FROM avg_2024
    FULL JOIN avg_2023 ON avg_2024.month = avg_2023.month
    ORDER BY month
    ''',
    with_column_types=True)

column_names = [column[0] for column in columns]
df = pd.DataFrame(data, columns=column_names)
display(df)

,month,avg_order_size_2024,avg_order_size_2023,difference
0,1,254.506480,255.164159,-0.657679
1,2,255.169323,253.808994,1.360328
2,3,255.057074,254.809477,0.247597
3,4,255.718120,254.719599,0.998522
4,5,254.876941,255.142092,-0.265151
5,6,255.464819,255.444229,0.020590
6,7,254.853804,254.602811,0.250993
7,8,255.051252,254.694861,0.356391
8,9,255.043652,255.748409,-0.704757
9,10,254.744101,254.876065,-0.131964


### 10 пользователей с наибольшим количеством заказов за последний год, и их средний размер заказа за последний месяц

In [25]:
data, columns = client.execute(
    '''
    WITH 
        orders_count AS ( -- количество заказов пользователей за последний год
            SELECT 
                user_id,
                COUNT(*) AS order_count
            FROM orders
            WHERE created_at >= now() - INTERVAL 1 YEAR
            GROUP BY user_id
            ORDER BY order_count DESC
            LIMIT 10
        ),
        
        average_order_size AS ( -- средний размер заказа для пользователей, у которых есть заказы за последний месяц
            SELECT
                o.user_id,
                AVG(oi.price * oi.quantity) AS avg_order_size
            FROM orders o
            JOIN order_items oi ON o.id = oi.order_id
            WHERE o.created_at >= now() - INTERVAL 1 MONTH
            GROUP BY o.user_id
        )
    
    SELECT 
        u.id AS user_id,
        u.name AS name,
        u.email AS email,
        oc.order_count AS order_count,
        aos.avg_order_size AS avg_order_size
    FROM users u
    JOIN orders_count oc ON u.id = oc.user_id
    LEFT JOIN average_order_size aos ON u.id = aos.user_id
    ORDER BY oc.order_count DESC;
    ''',
    with_column_types=True)

column_names = [column[0] for column in columns]
df = pd.DataFrame(data, columns=column_names)
display(df)

,user_id,name,email,order_count,avg_order_size
0,1066238999,CjAONDBJZy,cjaondbjzy@mail.ru,27,89057.415000
1,8258018890,spdGXEryBF,spdgxerybf@mail.ru,27,140789.312500
2,4402374827,FwUWuqAmce,fwuwuqamce@mail.ru,26,38807.020000
3,5136559090,ZlWgZoPrMZ,zlwgzoprmz@mail.ru,26,228740.976000
4,3134574026,twCGIvVoYN,twcgivvoyn@mail.ru,26,144855.308571
5,9447181256,sjwiCLRNSA,sjwiclrnsa@mail.ru,26,24876.660000
6,1594260174,lPRugtMrQs,lprugtmrqs@mail.ru,25,123710.220000
7,9325845087,PdRCUndMDa,pdrcundmda@mail.ru,25,248198.670000
8,6578425776,gCeohXkZCu,gceohxkzcu@mail.ru,25,99330.264000
9,8491657286,AvBssQdZkw,avbssqdzkw@mail.ru,25,188007.810000
